In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

# load from pkl file
def load_from_pkl(filename):
    return np.load(filename, allow_pickle=True)

data = load_from_pkl("../data/test.pkl")
print(data[1])

('Version 11r is the current version of the data set. Older versions will no longer be available and are superseded by Version 11r.\n\nThe Orbiting Carbon Observatory is the first NASA mission designed to collect space-based measurements of atmospheric carbon dioxide with the precision, resolution, and coverage needed to characterize the processes controlling its buildup in the atmosphere. The OCO-2 project uses the LEOStar-2 spacecraft that carries a single instrument. It incorporates three high-resolution spectrometers that make coincident measurements of reflected sunlight in the near-infrared CO2 near 1.61 and 2.06 micrometers and in molecular oxygen (O2) A-Band at 0.76 micrometers. The three spectrometers have different characteristics and are calibrated independently. Their raw data numbers (DN) are delivered correlated in time to the Level 1B process as Level 1A products. Each band has 1016 spectral elements, although some are masked out in the L2 retrieval.This product is the i

In [2]:
curl_cmd = """curl -X POST "https://gcmd.nasa-impact.net/api/requests/" -H  "accept: application/json" -H  "Content-Type: application/json" -H  "X-CSRFToken: Suj6XlUk5anVHNq6au0jpvDGxIoFJoUx9vgeplEhr1qrL5lYQG0NPriXZJm32v3u" -d "{  \"description\": \"{desc}\",  \"threshold\": 0,  \"forced\": false,  \"explain\": false}"""

import requests
import json

def gkr_post_request(desc):
    data = {
        "description": desc,
        "threshold": 0.75,
        "forced": False,
        "explain": False
    }
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "X-CSRFToken": "Suj6XlUk5anVHNq6au0jpvDGxIoFJoUx9vgeplEhr1qrL5lYQG0NPriXZJm32v3u"
    }
    resp = requests.post("https://gcmd.nasa-impact.net/api/requests/", data=json.dumps(data), headers=headers)
    resp_dict = json.loads(resp.text)
    return [r['keyword'] for r in resp_dict['recommendations']]

from gpt_embedder import GPTEmbedder
embedder = GPTEmbedder()

def top_n_accuracy(truth_list, predicted_list):
    """Calculate top N accuracy"""
    truth_list = [l.lower() for l in truth_list]
    predicted_list = [l.lower() for l in predicted_list]
    overlap = set(truth_list).intersection(set(predicted_list))
    return len(overlap) / len(truth_list)
        

In [3]:
data = load_from_pkl("../data/xy_data_list_GOG.p")

In [4]:
# data[0] contains all text
# data[1] contains all corresponding labels 
from tqdm import tqdm
gkr_acc = 0
gpt_acc = 0
for i in tqdm(range(len(data[0]))):
    try:
        # print(data[0][i])
        # print(data[1][i])
        gkr_predictions = [kw.split(">")[-1].strip() for kw in gkr_post_request(data[0][i])]
        gpt_precictions = [kw.split(">")[-1].strip() for kw in embedder.find_nearest_kw(data[0][i], top_n=5)]
        # print("GKR predictions: ", gkr_predictions)
        # print("GPT predictions: ", gpt_precictions)
        gkr_acc += top_n_accuracy(data[1][i], gkr_predictions)
        gpt_acc += top_n_accuracy(data[1][i], gpt_precictions)
    except:
        pass

100%|██████████| 3017/3017 [1:54:28<00:00,  2.28s/it]  


In [5]:
print("GKR accuracy: ", gkr_acc / len(data[0]))
print("GPT accuracy: ", gpt_acc / len(data[0]))

GKR accuracy:  0.3291617467636508
GPT accuracy:  0.17912072226271852


In [2]:
from gcmd_keyword_download import get_keyword_tree

text = get_keyword_tree()

ExpatError: not well-formed (invalid token): line 1, column 0

In [3]:
import requests
KEYWORD_URL = "https://gcmd.earthdata.nasa.gov/kms/tree/concept_scheme/all?version=15.9"
response = requests.get(KEYWORD_URL)
    

In [8]:
import json
gcmd_kwywords_file = 'gcmd_keywords.json'
json_data = json.load(open(gcmd_kwywords_file))


In [47]:
# json_data['tree']['treeData'][0]['children'][0]['children']...

# recursively get all keywords from the tree
keyword_path = ['']
def get_all_keywords(tree):
    keywords = []
    keyword_path[-1] += tree['title'] + ' > '
    for child in tree['children']:
        if child['children']:
            keywords += get_all_keywords(child, )
        else:
            keywords.append(child['title'])
            keyword_path.append('')
    return keywords


In [48]:
keywords = get_all_keywords(json_data['tree']['treeData'][0])


False

In [51]:
keywords

['AQUACULTURE',
 'FISHERIES',
 'ICE STUPA',
 'FERTILIZERS',
 'PESTICIDES',
 'AGRICULTURAL EQUIPMENT',
 'FARM STRUCTURES',
 'CROP/PLANT YIELDS',
 'CROPPING SYSTEMS',
 'IRRIGATION',
 'PLANT BREEDING AND GENETICS',
 'PLANT DISEASES/DISORDERS/PESTS',
 'RECLAMATION/REVEGETATION/RESTORATION',
 'WEEDS, NOXIOUS PLANTS OR INVASIVE PLANTS',
 'DAIRY PRODUCTS',
 'LIVESTOCK PRODUCTS',
 'POULTRY PRODUCTS',
 'ANIMAL BREEDING AND GENETICS',
 'ANIMAL DISEASES/DISORDERS/PESTS',
 'ANIMAL ECOLOGY AND BEHAVIOR',
 'ANIMAL MANAGEMENT SYSTEMS',
 'ANIMAL MANURE AND WASTE',
 'ANIMAL NUTRITION',
 'ANIMAL PHYSIOLOGY AND BIOCHEMISTRY',
 'ANIMAL YIELDS',
 'APICULTURE',
 'SERICULTURE',
 'FEED COMPOSITION',
 'FEED CONTAMINATION AND TOXICOLOGY',
 'FEED PROCESSING',
 'FEED STORAGE',
 'FOOD ADDITIVES',
 'FOOD CONTAMINATION AND TOXICOLOGY',
 'FOOD PACKAGING',
 'FOOD PROCESSING',
 'FOOD QUALITY',
 'FOOD STORAGE',
 'AFFORESTATION/REFORESTATION',
 'DEFOLIANTS',
 'FOREST CONSERVATION',
 'FOREST FIRE SCIENCE',
 'FOREST HARVES